In [1]:
!wget https://gist.githubusercontent.com/phillipj/4944029/raw/75ba2243dd5ec2875f629bf5d79f6c1e4b5a8b46/alice_in_wonderland.txt

--2024-11-19 14:16:03--  https://gist.githubusercontent.com/phillipj/4944029/raw/75ba2243dd5ec2875f629bf5d79f6c1e4b5a8b46/alice_in_wonderland.txt
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 148574 (145K) [text/plain]
Saving to: ‘alice_in_wonderland.txt’

alice_in_wonderland 100%[===================>] 145.09K  --.-KB/s    in 0.004s  

2024-11-19 14:16:03 (32.5 MB/s) - ‘alice_in_wonderland.txt’ saved [148574/148574]



In [2]:
!wget https://raw.githubusercontent.com/vifirsanova/hse-python-course/main/data/stopwords.txt

--2024-11-19 14:16:04--  https://raw.githubusercontent.com/vifirsanova/hse-python-course/main/data/stopwords.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 954 [text/plain]
Saving to: ‘stopwords.txt’

stopwords.txt       100%[===================>]     954  --.-KB/s    in 0s      

2024-11-19 14:16:04 (42.3 MB/s) - ‘stopwords.txt’ saved [954/954]



In [3]:
!pip install datasets -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [4]:
import re
import random
import spacy
from spacy import displacy
from pprint import pprint
import pandas as pd
from datasets import Dataset
from datasets import load_dataset

In [7]:
# Открыть и прочитать текстовый файл, записать в переменные
with open("alice_in_wonderland.txt", "r", encoding="utf-8") as file:
    aliceText = file.read()

with open("stopwords.txt", "r", encoding="utf-8") as  file:
    stopwords = file.read()

# Вывести на экран первые 100 элементов каждого объекта
print(f"Первые 100 символов текста 'Alice in Wonderland': {aliceText[:100]}")
print(f"\nПервые 100 стоп-слов: {stopwords.splitlines()[:100]}")

Первые 100 символов текста 'Alice in Wonderland': Alice's Adventures in Wonderland

                ALICE'S ADVENTURES IN WONDERLAND

                

Первые 100 стоп-слов: ['a', 'about', 'above', 'after', 'again', 'against', 'all', 'am', 'an', 'and', 'any', 'are', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', "can't", 'cannot', 'could', "couldn't", 'did', "didn't", 'do', 'does', "doesn't", 'doing', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', "hadn't", 'has', "hasn't", 'have', "haven't", 'having', 'he', "he'd", "he'll", "he's", 'her', 'here', "here's", 'hers', 'herself', 'him', 'himself', 'his', 'how', "how's", 'i', "i'd", "i'll", "i'm", "i've", 'if', 'in', 'into', 'is', "isn't", 'it', "it's", 'its', 'itself', "let's", 'me', 'more', 'most', "mustn't", 'my', 'myself', 'no', 'nor', 'not', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'ought', 'our', 'ours', 'ourselves', 'out']


In [49]:
def remove_punctuation(text):
    cleaned_text = re.sub(r'[^\w\s]', '', text)
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
    return cleaned_text

remove_punctuation('This is a sample      text! Wow :-)')

'This is a sample text Wow '

In [11]:
def remove_stopwords(text, stopwords):
    stopwords = stopwords.split("\n") # разбейте строку со стоп-словами на список слов
    words = text.split() # разбейте текст из переменной text на слова

    # если слово, приведенное к нижнему регистру присутствует в списке стоп-слов,
    # добавьте его в список filtered_words
    filtered_words = [word for word in words if word.lower() not in stopwords]
    return " ".join(filtered_words) # объедините слова из списка filtered_words в строку с join

# Протестировать функцию на тестовом образце
remove_stopwords('Below is the text without stopwords', stopwords)

'text without stopwords'

In [12]:
def tokenize_text(text):
    text_lower = text.lower() # привести к нижнему регистру текст
    tokenized_text = text_lower.split('\n') # разделить по переносу строки
    return tokenized_text

# Протестировать функцию на тестовом образце
tokenize_text('Tokenize this text\nWe\'ve got to tokenize it')

['tokenize this text', "we've got to tokenize it"]

In [13]:
tokenized_data = tokenize_text(aliceText)

# Вывести на экран тип объекта tokenized_data
print(f"Тип объекта tokenized_data: {type(tokenized_data)}")
# Вывести на экран количество элементов tokenized_data
print(f"В tokenized_data {len(tokenized_data)} элементов")
# Вывести на экран первые 100 элементов первого объекта tokenized_data
print(f"первые 100 символов первого элемента tokenized_data: {tokenized_data[0][:100]}")
# Вывести на экран первые 100 элементов второго объекта tokenized_data
print(f"первые 100 символов второго элемента tokenized_data: {tokenized_data[1][:100]}")

Тип объекта tokenized_data: <class 'list'>
В tokenized_data 3600 элементов
первые 100 символов первого элемента tokenized_data: alice's adventures in wonderland
первые 100 символов первого элемента tokenized_data: 


In [14]:
# примените функцию для очистки текстов от пунктуации к каждому элементу списка tokenized_data
# запишите результат в cleaned_data
cleaned_data = [remove_punctuation(line) for line in tokenized_data] # создайте список

# Вывести на экран первые 100 элементов первого объекта cleaned_data
print(f"Первые 100 символов первого элемента cleaned_data: {cleaned_data[0][:100]}")
# Вывести на экран первые 100 элементов второго объекта cleaned_data
print(f"Первые 100 символов второго элемента cleaned_data: {cleaned_data[1][:100]}")

Первые 100 символов первого элемента cleaned_data: alices adventures in wonderland
Первые 100 символов второго элемента cleaned_data: 


In [15]:
formatted_texts = []

for elem in cleaned_data: # для каждого элемента cleaned_data
    if len(elem) > 0: # если длина элемента > 0
        formatted_elem = f"<|reserved_special_token_0|> {elem} <|reserved_special_token_1|>"  # форматируйте текст с помощью f-strings для GPT
        formatted_texts.append(formatted_elem) # добавьте отформатированный элемент в список formatted_texts

formatted_texts[:100]

['<|reserved_special_token_0|> alices adventures in wonderland <|reserved_special_token_1|>',
 '<|reserved_special_token_0|>  alices adventures in wonderland <|reserved_special_token_1|>',
 '<|reserved_special_token_0|>  lewis carroll <|reserved_special_token_1|>',
 '<|reserved_special_token_0|>  the millennium fulcrum edition 30 <|reserved_special_token_1|>',
 '<|reserved_special_token_0|>  chapter i <|reserved_special_token_1|>',
 '<|reserved_special_token_0|>  down the rabbithole <|reserved_special_token_1|>',
 '<|reserved_special_token_0|>  alice was beginning to get very tired of sitting by her sister <|reserved_special_token_1|>',
 '<|reserved_special_token_0|> on the bank and of having nothing to do once or twice she had <|reserved_special_token_1|>',
 '<|reserved_special_token_0|> peeped into the book her sister was reading but it had no <|reserved_special_token_1|>',
 '<|reserved_special_token_0|> pictures or conversations in it and what is the use of a book <|reserved_special

In [16]:
# Сохраните результат в новый *.txt
with open("shakespeare_gpt2.txt", "w", encoding="utf-8") as file: # название файла shakespeare_gpt2 + режим для записи
    file.write("\n".join(formatted_texts)) # используйте join и разделитель \n

# **Лингвистический анализ**

1. Небольшая выборка для аннотирования данных

In [27]:
filtered_data = [] # если длина текста из cleaned_data > 0, добавляем его в список filtered_data
for elem in cleaned_data:
  if len(elem) > 0:
    filtered_data.append(elem)

sample_size = int(len(filtered_data))  # задаем размер выборки: 10%
sample_data = random.sample(filtered_data, sample_size) # семплирование
sample_text = ' '.join(sample_data) # объедините все в одну строку
sample_text

' no no the adventures first said the gryphon in an whatever happens what will become of me thats about the right distancebut then i wonder what latitude people that walk with their heads downward the antipathies i from his book rule fortytwo all persons more than a mile the march hare and his friends shared their neverending meal what are you thinking of  beautiful soup who cares for fish trying to box her own ears for having cheated herself in a game so much about a whiting before doesnt begin but she waited patiently angrily rearing itself upright as it spoke it was exactly three andbutter  law i will pardon your majesty he began for bringing these in but i grunt said alice thats not at all a proper way of expressing of little birds and beasts as well as the whole pack of cards politely did you speak even know what theyre about  alice said nothing she had never been so much contradicted in  i thought it would said the cat and vanished again angry and wags its tail when its pleased n

2.1. Лемматизация (семпл)

In [ ]:
nlp = spacy.load("en_core_web_sm") # подгружаем модель

doc = nlp(sample_text)

lemmas = []

for token in doc:
  lemmas.append(token.lemma_)

for i in range(30): # выведите пары (token, lemma) в диапазоне 30,
  print(f'TOKEN: {doc[i]}, LEMMA: {lemmas[i]}, ITERATION: {i}') # т.е. для первых 30 элементов списка lemmas и для первых 30 элементов doc

2.2. Лемматизация (полный текст)

In [36]:
full_text = ' '.join(cleaned_data)  # объединение всех строк в один текст

nlp = spacy.load("en_core_web_sm")
doc = nlp(full_text)

lemmas = [token.lemma_ for token in doc]

for i in range(30): # выведите пары (token, lemma) в диапазоне 30,
    print(f'TOKEN: {doc[i]}, LEMMA: {lemmas[i]}, ITERATION: {i}') # т.е. для первых 30 элементов списка lemmas и для первых 30 элементов doc

TOKEN: alices, LEMMA: alice, ITERATION: 0
TOKEN: adventures, LEMMA: adventure, ITERATION: 1
TOKEN: in, LEMMA: in, ITERATION: 2
TOKEN: wonderland, LEMMA: wonderland, ITERATION: 3
TOKEN:   , LEMMA:   , ITERATION: 4
TOKEN: alices, LEMMA: alice, ITERATION: 5
TOKEN: adventures, LEMMA: adventure, ITERATION: 6
TOKEN: in, LEMMA: in, ITERATION: 7
TOKEN: wonderland, LEMMA: wonderland, ITERATION: 8
TOKEN:   , LEMMA:   , ITERATION: 9
TOKEN: lewis, LEMMA: lewis, ITERATION: 10
TOKEN: carroll, LEMMA: carroll, ITERATION: 11
TOKEN:   , LEMMA:   , ITERATION: 12
TOKEN: the, LEMMA: the, ITERATION: 13
TOKEN: millennium, LEMMA: millennium, ITERATION: 14
TOKEN: fulcrum, LEMMA: fulcrum, ITERATION: 15
TOKEN: edition, LEMMA: edition, ITERATION: 16
TOKEN: 30, LEMMA: 30, ITERATION: 17
TOKEN:      , LEMMA:      , ITERATION: 18
TOKEN: chapter, LEMMA: chapter, ITERATION: 19
TOKEN: i, LEMMA: I, ITERATION: 20
TOKEN:   , LEMMA:   , ITERATION: 21
TOKEN: down, LEMMA: down, ITERATION: 22
TOKEN: the, LEMMA: the, ITERATION:

3. Частеречная разметка

In [37]:
pos_annotation = []

for token in doc:
  pos_annotation.append((token.text, token.pos_))

for i in range(30):
  print(f'{pos_annotation[i][0]} -> {pos_annotation[i][1]}') # выведите пары (token, pos) в диапазоне 30

alices -> NOUN
adventures -> NOUN
in -> ADP
wonderland -> NOUN
   -> SPACE
alices -> NOUN
adventures -> NOUN
in -> ADP
wonderland -> NOUN
   -> SPACE
lewis -> PROPN
carroll -> NOUN
   -> SPACE
the -> DET
millennium -> NOUN
fulcrum -> NOUN
edition -> NOUN
30 -> NUM
      -> SPACE
chapter -> NOUN
i -> PRON
   -> SPACE
down -> ADP
the -> DET
rabbithole -> NOUN
    -> SPACE
alice -> PROPN
was -> AUX
beginning -> VERB
to -> PART


4. Синтаксическая разметка

In [38]:
doc_ = nlp('everybody looked at alice')

syntax_annotation = [(token.text, token.dep_, token.head.text) for token in doc_]
displacy.render(doc_, style="dep", jupyter=True, options={'distance': 150})

5. Разметка Named Entity Recognition

In [39]:
ner_annotation = [(ent.text, ent.label_) for ent in doc.ents]
displacy.render(doc, style="ent", jupyter=True)

6. Создайте словарь с результатами разметки

In [50]:
# словарь вида
results = dict() # исходный текст
results['Lemmatized Text'] = lemmas[:10] # леммы
results['POS Annotation'] = pos_annotation[:10] # части речи
# results['NER Annotation'] = ner_annotation[:10] # именованные сущности

pprint(results)

{'Lemmatized Text': ['alice',
                     'adventure',
                     'in',
                     'wonderland',
                     '  ',
                     'alice',
                     'adventure',
                     'in',
                     'wonderland',
                     '  '],
 'POS Annotation': [('alices', 'NOUN'),
                    ('adventures', 'NOUN'),
                    ('in', 'ADP'),
                    ('wonderland', 'NOUN'),
                    ('  ', 'SPACE'),
                    ('alices', 'NOUN'),
                    ('adventures', 'NOUN'),
                    ('in', 'ADP'),
                    ('wonderland', 'NOUN'),
                    ('  ', 'SPACE')]}


In [41]:
df = pd.DataFrame(results) # создаем датафрейм с результатами
df.head()

,Lemmatized Text,POS Annotation
0,alice,"(alices, NOUN)"
1,adventure,"(adventures, NOUN)"
2,in,"(in, ADP)"
3,wonderland,"(wonderland, NOUN)"
4,,"( , SPACE)"


In [42]:
df['POS Annotation'][0][:10]

('alices', 'NOUN')

In [43]:
print('token ->', df['POS Annotation'][0][0], '\nlabel ->', df['POS Annotation'][0][1])

token -> alices 
label -> NOUN


Сохраняем в *.csv

In [44]:
df.to_csv('nlp_results.csv', index=False)

**Загружаем датасет на HuggingFace**

In [45]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
The token `datasets` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `datasets`


In [46]:
ds = Dataset.from_dict(results)

In [47]:
ds

Dataset({
    features: ['Lemmatized Text', 'POS Annotation'],
    num_rows: 27854
})

In [48]:
ds.push_to_hub("veriuzhskii/alice_in_wonderland")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/28 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/358 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/veriuzhskii/alice_in_wonderland/commit/aacf6d6b87b9a7b52c8bfd1ce3c8cd2aa4ee0c4c', commit_message='Upload dataset', commit_description='', oid='aacf6d6b87b9a7b52c8bfd1ce3c8cd2aa4ee0c4c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/veriuzhskii/alice_in_wonderland', endpoint='https://huggingface.co', repo_type='dataset', repo_id='veriuzhskii/alice_in_wonderland'), pr_revision=None, pr_num=None)